In [9]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### MNISTのデータセットを作成

In [10]:
# as_supervised=True：入力のデータと正解データをタプルにして返す
# with_info=True：データセットの情報を返す (mnist_info)
mnist_dataset, mnist_info = tfds.load(name="mnist", with_info=True, as_supervised=True)

訓練用とテスト用のデータセットを作成する。

In [17]:
mnist_train, mnist_test = mnist_dataset["train"], mnist_dataset["test"]

# 今回は訓練用データをデータセットの10%とする
num_validation_samples = 0.1 * mnist_info.splits["train"].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)


# データに型付けする
num_test_samples = mnist_info.splits["test"].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

print(num_validation_samples)

AttributeError: module 'tensorflow' has no attribute 'int'

標準化する (具体的には0 ~ 255の値を0 ~ 1の値に変換する)


In [11]:
def scale(image, label):
    """
    画像データのピクセル値を0~1にスケーリングする
    """
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

データシャッフル

In [20]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(
    BUFFER_SIZE
)
#  take()：指定した数の要素を取得する
validation_data = shuffled_train_and_validation_data.take(int(num_validation_samples))
# skip()：指定した数の要素をスキップする,
train_data = shuffled_train_and_validation_data.skip(int(num_validation_samples))

BATCH_SIZE = 100

# データをバッチサイズごとに分割する
# バックプロパゲーションの処理は重いため
train_data = train_data.batch(BATCH_SIZE)

# バリデーションとテストデータは一度に読み込む、フォワードプロパゲーションのみ行うため
validation_data = validation_data.batch(int(num_validation_samples))
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

2024-01-06 23:23:00.094351: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### モデル作成

In [ ]:
input_size = 784
output_size = 10
hidden_layer_size = 50

# tf.keras.Sequential()：モデルを作成する
model = tf.keras.Sequential(
    [
        # tf.keras.layers.Flatten()：入力データを1次元に変換する
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        # tf.keras.layers.Dense()：全結合層を作成する
        # units：出力の次元数
        # activation：活性化関数
        tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
        tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
        tf.keras.layers.Dense(output_size, activation="softmax"),
    ]
)

### 最適化アルゴリズムと損失関数を設定

In [24]:
# tf.keras.optimizers.Adam()：Adamアルゴリズムを使用してモデルを最適化する
# loss：損失関数
# sparse_categorical_crossentropyとは：https://qiita.com/omiita/items/0b7d4c0c6d5d4c4b0b0e
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

訓練

In [29]:
# 繰り返した回数
NUM_EPOCHS = 5

VALIDATION_STEPS = num_validation_samples

# モデルを訓練する
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), validation_steps=VALIDATION_STEPS, verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.0149 - accuracy: 0.9956 - val_loss: 0.0248 - val_accuracy: 0.9933 - 3s/epoch - 5ms/step
Epoch 2/5
540/540 - 3s - loss: 0.0143 - accuracy: 0.9957 - val_loss: 0.0271 - val_accuracy: 0.9918 - 3s/epoch - 5ms/step
Epoch 3/5
540/540 - 3s - loss: 0.0125 - accuracy: 0.9960 - val_loss: 0.0205 - val_accuracy: 0.9952 - 3s/epoch - 5ms/step
Epoch 4/5
540/540 - 3s - loss: 0.0094 - accuracy: 0.9975 - val_loss: 0.0188 - val_accuracy: 0.9942 - 3s/epoch - 5ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0136 - accuracy: 0.9957 - val_loss: 0.0427 - val_accuracy: 0.9868 - 2s/epoch - 4ms/step


### モデルのテスト

In [30]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 224ms/step - loss: 0.1475 - accuracy: 0.9697


損失関数と精度を表示

In [31]:
print("Test loss: {0:.2f}. Test accuracy: {1:.2f}%".format(test_loss, test_accuracy * 100.0))

Test loss: 0.15. Test accuracy: 96.97%
